# Fix all jupyter notebook problems

<sub> 0. Pray </sub> 
1. Connect to head machine via SSH
2. Open `/usr/bin/anaconda/lib/python2.7/site-packages/nbformat/_version.py` and change 5 to 4.
3. Fix anaconda installation via official fix script. 
```
curl https://gregorysfixes.blob.core.windows.net/public/fix-conda.sh | sudo sh
```
4. Install all necessary python packages. At least kaggle - 
```
sudo /usr/bin/anaconda/bin/conda install -c conda-forge kaggle --yes
```
5. Open Ambari and restart jupyter service.
6. Open azure jupyter notebook and upload this notebook
7. Check, that cells below can be executed correctly

# Create Spark Context

In [0]:
sc = spark.sparkContext

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1584535917646_0011,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [0]:
import pandas as pd
from pyspark.sql import SparkSession

ss = SparkSession(sc)

In [0]:
hadoop = sc._jvm.org.apache.hadoop
fs = hadoop.fs.FileSystem
conf = hadoop.conf.Configuration() 
path = hadoop.fs.Path('/')
    
def hdfs_ls(path):
    result = []
    for f in fs.get(conf).listStatus(hadoop.fs.Path(path)):
        result.append(str(f.getPath()))
    return result

In [0]:
from pprint import pprint

pprint(hdfs_ls('/'))

['wasb://sumekenov-2020-03-18t12-37-25-691z@sumekenovhdistorage.blob.core.windows.net/custom-scriptaction-logs',
 'wasb://sumekenov-2020-03-18t12-37-25-691z@sumekenovhdistorage.blob.core.windows.net/app-logs',
 'wasb://sumekenov-2020-03-18t12-37-25-691z@sumekenovhdistorage.blob.core.windows.net/tmp',
 'wasb://sumekenov-2020-03-18t12-37-25-691z@sumekenovhdistorage.blob.core.windows.net/warehouse',
 'wasb://sumekenov-2020-03-18t12-37-25-691z@sumekenovhdistorage.blob.core.windows.net/mapred',
 'wasb://sumekenov-2020-03-18t12-37-25-691z@sumekenovhdistorage.blob.core.windows.net/HDInsight_TestAccessiblityBlobName',
 'wasb://sumekenov-2020-03-18t12-37-25-691z@sumekenovhdistorage.blob.core.windows.net/hive',
 'wasb://sumekenov-2020-03-18t12-37-25-691z@sumekenovhdistorage.blob.core.windows.net/example',
 'wasb://sumekenov-2020-03-18t12-37-25-691z@sumekenovhdistorage.blob.core.windows.net/apps',
 'wasb://sumekenov-2020-03-18t12-37-25-691z@sumekenovhdistorage.blob.core.windows.net/HdiSamples',
 

# Download task data

Download data directly from kaggle. Read this to understand how: https://github.com/Kaggle/kaggle-api

In [0]:
%%local
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/home/spark/.kaggle’: File exists


In [0]:
%%local
! rm ~/.kaggle/kaggle.json

In [0]:
%%local
! touch ~/.kaggle/kaggle.json

In [0]:
%%local
! echo '{"username":"ahmad2520","key":"753719b27680c36fbafe26f9b5d12e31"}' >> ~/.kaggle/kaggle.json

In [0]:
%%local
! cat ~/.kaggle/kaggle.json

{"username":"ahmad2520","key":"753719b27680c36fbafe26f9b5d12e31"}


In [0]:
%%local
! kaggle competitions files outbrain-click-prediction

name                           size  creationDate         
----------------------------  -----  -------------------  
clicks_test.csv.zip           135MB  2018-06-22 05:33:10  
sample_submission.csv.zip     100MB  2018-06-22 05:33:10  
documents_entities.csv.zip    126MB  2018-06-22 05:33:10  
documents_meta.csv.zip         16MB  2018-06-22 05:33:10  
documents_topics.csv.zip      121MB  2018-06-22 05:33:10  
events.csv.zip                478MB  2018-06-22 05:33:10  
page_views.csv.zip             35GB  2018-06-22 05:33:10  
clicks_train.csv.zip          390MB  2018-06-22 05:33:10  
documents_categories.csv.zip   32MB  2018-06-22 05:33:10  
promoted_content.csv.zip        3MB  2018-06-22 05:33:10  
page_views_sample.csv.zip     149MB  2018-06-22 05:33:10  


In [0]:
%%local
! kaggle competitions download outbrain-click-prediction

404 - Not Found


In [0]:
%%local
!for f in $(kaggle competitions files outbrain-click-prediction | cut -d ' ' -f 1 | tail -n+3); do kaggle competitions download --force outbrain-click-prediction -f $f; done

usage: kaggle competitions download [-h] [-f FILE_NAME] [-p PATH] [-w] [-o]
                                    [-q]
                                    [competition]
kaggle competitions download: error: argument -f/--file: expected one argument
 90%|████████▉ | 29.0M/32.3M [00:00<00:00, 72.1MB/s]
100%|██████████| 32.3M/32.3M [00:00<00:00, 92.8MB/s]
 89%|████████▉ | 121M/135M [00:02<00:00, 57.4MB/s] 
100%|██████████| 135M/135M [00:02<00:00, 50.3MB/s]
  0%|          | 0.00/2.52M [00:00<?, ?B/s]
100%|██████████| 2.52M/2.52M [00:00<00:00, 41.2MB/s]
 90%|█████████ | 90.0M/99.6M [00:01<00:00, 67.8MB/s]
100%|██████████| 99.6M/99.6M [00:01<00:00, 66.6MB/s]
 99%|█████████▉| 386M/390M [00:03<00:00, 159MB/s]  
100%|██████████| 390M/390M [00:03<00:00, 107MB/s]
100%|██████████| 149M/149M [00:04<00:00, 38.2MB/s] 
100%|██████████| 149M/149M [00:04<00:00, 31.3MB/s]
 58%|█████▊    | 9.00M/15.5M [00:00<00:00, 25.2MB/s]
100%|██████████| 15.5M/15.5M [00:00<00:00, 39.2MB/s]
100%|█████████▉| 34.8G/34.8G [1

# Load data to HDFS

https://www.kaggle.com/c/outbrain-click-prediction/data

In [0]:
%%local
! hdfs dfs -rm -r /task1
! hdfs dfs -mkdir /task1

rm: `/task1': No such file or directory


In [0]:
%%local
! for i in `ls *.zip`; do unzip -p $i | tqdm | hadoop fs -put - /task1/${i//\.zip/}; done

32225163it [00:48, 663902.27it/s]
87141732it [02:10, 669204.74it/s]
5481476it [00:09, 557283.10it/s]
5537553it [00:13, 410678.52it/s]
2999335it [00:06, 466028.75it/s]
11325961it [00:19, 580775.42it/s]
23120127it [00:41, 553977.32it/s]
2034275449it [58:06, 583465.02it/s]
10000000it [00:19, 504750.56it/s]
559584it [00:02, 217890.17it/s]
6245534it [00:13, 477228.81it/s]


In [0]:
%%local
! hadoop fs -du -s -h /task1/*.csv

483.5 M  483.5 M  /task1/clicks_test.csv
1.4 G    1.4 G    /task1/clicks_train.csv
112.5 M  112.5 M  /task1/documents_categories.csv
309.1 M  309.1 M  /task1/documents_entities.csv
85.2 M   85.2 M   /task1/documents_meta.csv
323.7 M  323.7 M  /task1/documents_topics.csv
1.1 G    1.1 G    /task1/events.csv
88.4 G   88.4 G   /task1/page_views.csv
433.3 M  433.3 M  /task1/page_views_sample.csv
13.2 M   13.2 M   /task1/promoted_content.csv
260.5 M  260.5 M  /task1/sample_submission.csv


# Read example

In [0]:
pvdf = ss.read.csv("/task1/page_views.csv", header=True)

In [0]:
pvdf.dtypes

[('uuid', 'string'), ('document_id', 'string'), ('timestamp', 'string'), ('platform', 'string'), ('geo_location', 'string'), ('traffic_source', 'string')]

In [0]:
pvdf.show(5)

+--------------+-----------+---------+--------+------------+--------------+
|          uuid|document_id|timestamp|platform|geo_location|traffic_source|
+--------------+-----------+---------+--------+------------+--------------+
|1fd5f051fba643|        120| 31905835|       1|          RS|             2|
|8557aa9004be3b|        120| 32053104|       1|       VN>44|             2|
|c351b277a358f0|        120| 54013023|       1|       KR>12|             1|
|8205775c5387f9|        120| 44196592|       1|       IN>16|             2|
|9cb0ccd8458371|        120| 65817371|       1|   US>CA>807|             2|
+--------------+-----------+---------+--------+------------+--------------+
only showing top 5 rows

In [0]:
pvdf.count()

2034275448

# Parquet is faster than CSV

http://events.linuxfoundation.org/sites/events/files/slides/ApacheCon%20BigData%20Europe%202016%20-%20Parquet%20in%20Practice%20%26%20Detail_0.pdf

# Convert all to Parquet

In [0]:
def convert_all_to_parquet():
    task_dir = "/task1/"
    all_files = hdfs_ls(task_dir)
    for fn in all_files:
        if fn.endswith(".csv"):
            fn_after = fn.replace(".csv", ".parquet")
            path_before = fn
            path_after = fn_after
            if fn_after not in all_files:
                # generate parquet
                df = ss.read.csv(path_before, header=True)
                df.write.parquet(path_after)
            print(fn_after, "done")

convert_all_to_parquet()

Starting Spark application


KeyboardInterrupt: 

Remove csv, we have parquet now

In [0]:
%%local
! hdfs dfs -rm /task1/*.csv

Deleted /task1/clicks_test.csv
Deleted /task1/clicks_train.csv
Deleted /task1/documents_categories.csv
Deleted /task1/documents_entities.csv
Deleted /task1/documents_meta.csv
Deleted /task1/documents_topics.csv
Deleted /task1/events.csv
Deleted /task1/page_views.csv
Deleted /task1/page_views_sample.csv
Deleted /task1/promoted_content.csv
Deleted /task1/sample_submission.csv


In [0]:
%%local
! hadoop fs -du -s -h /task1/*

110.0 M  110.0 M  /task1/clicks_test.parquet
324.3 M  324.3 M  /task1/clicks_train.parquet
34.1 M   34.1 M   /task1/documents_categories.parquet
195.9 M  195.9 M  /task1/documents_entities.parquet
21.1 M   21.1 M   /task1/documents_meta.parquet
181.5 M  181.5 M  /task1/documents_topics.parquet
685.6 M  685.6 M  /task1/events.parquet
47.4 G   47.4 G   /task1/page_views.parquet
236.9 M  236.9 M  /task1/page_views_sample.parquet
5.0 M    5.0 M    /task1/promoted_content.parquet
184.4 M  184.4 M  /task1/sample_submission.parquet
20/03/18 15:59:58 WARN impl.MetricsSinkAdapter: azurefs2 has a full queue and can't consume the given metrics.


# Preview all files

In [0]:
# %%time
def preview_all_files():
    task_dir = "/task1/"
    all_files = hdfs_ls(task_dir)
    for fn in all_files:
        df = ss.read.parquet(fn)
        print("#" * 15 + " {0} ".format(fn) + "#" * 15)
        df.show(1)
        
preview_all_files()

'Unable to infer schema for Parquet. It must be specified manually.;'
Traceback (most recent call last):
  File "<stdin>", line 6, in preview_all_files
  File "/usr/hdp/current/spark2-client/python/pyspark/sql/readwriter.py", line 316, in parquet
    return self._df(self._jreader.parquet(_to_seq(self._spark._sc, paths)))
  File "/usr/hdp/current/spark2-client/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/hdp/current/spark2-client/python/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'Unable to infer schema for Parquet. It must be specified manually.;'



# Register all tables to be usable in SQL queries

In [0]:
def register_all_tables():
    task_dir = "/task1/"
    all_files = hdfs_ls(task_dir)
    for fn in all_files:
        if fn.endswith(".parquet"):
            table_name = fn.replace(".parquet", "")
            df = ss.read.parquet(fn)
            df.registerTempTable(table_name.split('/')[-1])
            print(table_name, "done")
        
register_all_tables()

name 'hdfs_ls' is not defined
Traceback (most recent call last):
  File "<stdin>", line 3, in register_all_tables
NameError: name 'hdfs_ls' is not defined



In [0]:
ss.sql("SHOW TABLES").show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| default|     hivesampletable|      false|
|        |         clicks_test|       true|
|        |        clicks_train|       true|
|        |documents_categories|       true|
|        |  documents_entities|       true|
|        |      documents_meta|       true|
|        |    documents_topics|       true|
|        |              events|       true|
|        |          page_views|       true|
|        |   page_views_sample|       true|
|        |    promoted_content|       true|
|        |   sample_submission|       true|
|        |                test|       true|
|        |               train|       true|
+--------+--------------------+-----------+

# SQL query example

In [0]:
ss.sql("""
select count(distinct(uuid)) as users_count
from events
""").collect()

[Row(users_count=19794967)]

# 1. Baseline

Simple model using the following features:
- **clicked**
- geo_location features (country, state, dma)
- day_of_week (from timestamp, use *date.isoweekday()*)
- ad_id
- ad_document_id
- campaign_id
- advertiser_id
- display_document_id
- platform

## Calculate features for VW

- Use DataFrame API to join tables (functions in SQL queries: https://spark.apache.org/docs/2.1.0/api/java/org/apache/spark/sql/functions.html)
- Use Python API to calculate features and save them as text for VW (*saveAsTextFile()*)
- Hash features in Spark (24 bits, use *sklearn.utils.murmurhash.murmurhash3_32*)
- Split dataset in Spark into 90% train, 10% test **by display_id**, save the split for further use

In [0]:
from sklearn.utils.murmurhash import murmurhash3_32
def hasher(x, bits):
    return murmurhash3_32(x) % 2**bits

In [0]:
clicks_train = ss.sql('SELECT * FROM clicks_train')
print(clicks_train.columns)

['display_id', 'ad_id', 'clicked']

In [0]:
documents_categories = ss.sql('SELECT * FROM documents_categories')
print(documents_categories.columns)

['document_id', 'category_id', 'confidence_level']

In [0]:
sample_submission = ss.sql('SELECT * FROM sample_submission')
print(sample_submission.columns)

In [0]:
events = ss.sql("SELECT * FROM events")
print(events.columns)

['display_id', 'uuid', 'document_id', 'timestamp', 'platform', 'geo_location']

In [0]:
page_views = ss.sql('SELECT * FROM page_views')
print(page_views.columns)

['uuid', 'document_id', 'timestamp', 'platform', 'geo_location', 'traffic_source']

In [0]:
promoted_content = ss.sql('SELECT * FROM promoted_content')
print(promoted_content.columns)

['ad_id', 'document_id', 'campaign_id', 'advertiser_id']

In [0]:
clicks_test = ss.sql('SELECT * FROM clicks_test')
print(clicks_test.columns)

['display_id', 'ad_id']

In [0]:
clicks_train = clicks_train.withColumnRenamed('display_id', 'clicks_display_id')
superTable = events.join(clicks_train, clicks_train.clicks_display_id==events.display_id).drop('clicks_display_id')
superTable.show(10)

+----------+--------------+-----------+---------+--------+------------+------+-------+
|display_id|          uuid|document_id|timestamp|platform|geo_location| ad_id|clicked|
+----------+--------------+-----------+---------+--------+------------+------+-------+
|  10000108|66d17a9ccd0e15|    1126487|668707895|       1|   US>CA>807|132815|      1|
|  10000108|66d17a9ccd0e15|    1126487|668707895|       1|   US>CA>807|133677|      0|
|  10000108|66d17a9ccd0e15|    1126487|668707895|       1|   US>CA>807|406686|      0|
|  10000108|66d17a9ccd0e15|    1126487|668707895|       1|   US>CA>807|406704|      0|
|  10000108|66d17a9ccd0e15|    1126487|668707895|       1|   US>CA>807|449087|      0|
|  10000108|66d17a9ccd0e15|    1126487|668707895|       1|   US>CA>807|471551|      0|
|  10000108|66d17a9ccd0e15|    1126487|668707895|       1|   US>CA>807|488469|      0|
|  10000172|11c89d2bcfd60b|    1233989|668711251|       1|   US>OK>671| 45192|      0|
|  10000172|11c89d2bcfd60b|    1233989|6687

In [0]:
# (23120126, 87141731)


In [0]:
events.count(), superTable.count()

(23120126, 87141731)

In [0]:
promoted_content = promoted_content.withColumnRenamed('ad_id', 'promo_ad_id')
promoted_content = promoted_content.withColumnRenamed('document_id', 'promo_document_id')
superTable = superTable.join(promoted_content, superTable.ad_id==promoted_content.promo_ad_id).drop('promo_ad_id')
superTable.show(10)

+----------+--------------+-----------+---------+--------+------------+------+-------+-----------------+-----------+-------------+
|display_id|          uuid|document_id|timestamp|platform|geo_location| ad_id|clicked|promo_document_id|campaign_id|advertiser_id|
+----------+--------------+-----------+---------+--------+------------+------+-------+-----------------+-----------+-------------+
|  10000108|66d17a9ccd0e15|    1126487|668707895|       1|   US>CA>807|132815|      1|          1227645|      17018|          331|
|  10000108|66d17a9ccd0e15|    1126487|668707895|       1|   US>CA>807|133677|      0|          1297868|      17143|         1919|
|  10000108|66d17a9ccd0e15|    1126487|668707895|       1|   US>CA>807|406686|      0|          1671134|      27505|         2879|
|  10000108|66d17a9ccd0e15|    1126487|668707895|       1|   US>CA>807|406704|      0|          1678714|      27506|         2879|
|  10000108|66d17a9ccd0e15|    1126487|668707895|       1|   US>CA>807|449087|     

In [0]:
superTable.count()

87141731

In [0]:
clicks_test = clicks_test.withColumnRenamed('display_id', 'clicks_display_id')
superTableTest = events.join(clicks_test, clicks_test.clicks_display_id==events.display_id).drop('clicks_display_id')
promoted_content = promoted_content.withColumnRenamed('ad_id', 'promo_ad_id')
promoted_content = promoted_content.withColumnRenamed('document_id', 'promo_document_id')
superTableTest = superTableTest.join(promoted_content, superTableTest.ad_id==promoted_content.promo_ad_id).drop('promo_ad_id')

In [0]:
superTableTest.count()

32225162

In [0]:
superTable.coalesce(1).write.mode('overwrite').parquet('/task1/train.parquet')

In [0]:
superTableTest.write.mode('overwrite').parquet('/task1/test.parquet')

In [0]:
superTableTest.count()

32225162

In [0]:
superTable.columns

['display_id', 'uuid', 'document_id', 'timestamp', 'platform', 'geo_location', 'ad_id', 'clicked', 'promo_document_id', 'campaign_id', 'advertiser_id']

In [0]:
from datetime import date
template = "{label} |f did{display_id} uid{uuid} doid{document_id} dow{day_of_the_week} prm{platform} adid{ad_id} pdid{promo_document_id} cid{campaing_id} advid{advertiser_id} {geo}"
def vw_train(row):
    geo = 'NULLGEO NULLGEO NULLGEO'
    if row.geo_location and '>' in row.geo_location:
        locations = row.geo_location.split('>')
        if len(locations) == 1:
            geo = locations[0] + ' NULLGEO NULLGEO'
        elif len(locations) == 2:
            geo = locations[0] + ' ' + locations[1] + ' ' + 'NULLGEO'
        else:
            geo = locations[0] + ' ' + locations[1] + ' ' + locations[2]
    return template.format(
        label = 1 if int(row.clicked) == 1 else -1, 
        display_id = row.display_id,
        uuid = row.uuid, 
        document_id = row.document_id,
        day_of_the_week = date.isoweekday(date.fromtimestamp(int(row.timestamp))),
        platform = row.platform,
        ad_id = row.ad_id,
        promo_document_id = row.promo_document_id,
        campaing_id = row.campaign_id,
        advertiser_id = row.advertiser_id,
        geo=geo
    )

In [0]:
template = '|f did{display_id} uid{uuid} doid{document_id} dow{day_of_the_week} prm{platform} adid{ad_id} pdid{promo_document_id} cid{campaing_id} advid{advertiser_id} {geo}"'
def vw_test(row):
    geo = 'NULLGEO NULLGEO NULLGEO'
    if row.geo_location and '>' in row.geo_location:
        locations = row.geo_location.split('>')
        if len(locations) == 1:
            geo = locations[0] + ' NULLGEO NULLGEO'
        elif len(locations) == 2:
            geo = locations[0] + ' ' + locations[1] + ' ' + 'NULLGEO'
        else:
            geo = locations[0] + ' ' + locations[1] + ' ' + locations[2]
    return template.format(
        display_id = row.display_id,
        uuid = row.uuid, 
        document_id = row.document_id,
        day_of_the_week = date.isoweekday(date.fromtimestamp(int(row.timestamp))),
        platform = row.platform,
        ad_id = row.ad_id,
        promo_document_id = row.promo_document_id,
        campaing_id = row.campaign_id,
        advertiser_id = row.advertiser_id,
        geo=geo
    )

In [0]:
%%local
! hdfs dfs -rm -r /task1/train.txt

20/03/18 22:34:45 WARN azure.AzureFileSystemThreadPoolExecutor: Disabling threads for Delete operation as thread count 0 is <= 1
20/03/18 22:34:45 WARN azure.NativeAzureFileSystem: Attempt to delete non-existent directory task1/train.txt/_temporary
20/03/18 22:34:45 INFO azure.AzureFileSystemThreadPoolExecutor: Time taken for Delete operation is: 38 ms with threads: 0
Deleted /task1/train.txt


In [0]:
superTable.rdd.map(vw_train).saveAsTextFile("/task1/train.txt")

In [0]:
%%local
! hdfs dfs -rm -r /task1/test.txt

20/03/18 23:25:21 WARN azure.AzureFileSystemThreadPoolExecutor: Disabling threads for Delete operation as thread count 0 is <= 1
20/03/18 23:25:23 INFO azure.AzureFileSystemThreadPoolExecutor: Time taken for Delete operation is: 1909 ms with threads: 0
Deleted /task1/test.txt


In [0]:
superTableTest.rdd.map(vw_test).saveAsTextFile("/task1/test.txt")

## Copy data from HDFS to cluster machine

We will run vowpal wabbit **locally**, need to copy data from HDFS

In [0]:
%%local
! hdfs dfs -getmerge /task1/train.txt train.txt

In [0]:
%%local
! hdfs dfs -getmerge /task1/test.txt test.txt

## Install VW

Connect to head node via SSH and install vw:

```
git clone --recursive https://github.com/VowpalWabbit/vowpal_wabbit.git
sudo apt install libboost-dev libboost-program-options-dev libboost-system-dev libboost-thread-dev libboost-math-dev libboost-test-dev zlib1g-dev cmake g++ -y
cd vowpal_wabbit && make && cd build && make install
echo "export PATH=/usr/local/bin:\$PATH" >> ~/.bashrc
```

Or you can download prebuilded vw 
```
mkdir -p bin/
wget http://finance.yendor.com/ML/VW/Binaries/vw-8.20190624 -O bin/vw
chmod +x bin/vw
```

In [0]:
%%local
! wget http://finance.yendor.com/ML/VW/Binaries/vw-8.20190624 -O vw
! chmod +x vw

--2020-03-18 21:13:45--  http://finance.yendor.com/ML/VW/Binaries/vw-8.20190624
Resolving finance.yendor.com (finance.yendor.com)... 69.163.152.190
Connecting to finance.yendor.com (finance.yendor.com)|69.163.152.190|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9467376 (9.0M)
Saving to: ‘vw’

vw                  100%[===================>]   9.03M  11.3MB/s    in 0.8s    

2020-03-18 21:13:46 (11.3 MB/s) - ‘vw’ saved [9467376/9467376]



In [0]:
%%local
! ./vw --help | head

Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = 
num sources = 1
driver:
  --onethread           Disable parse thread
VW options:
  --ring_size arg (=256, ) size of example ring
  --strict_parse           throw on malformed examples
Update options:
  -l [ --learning_rate ] arg Set learning rate
  --power_t arg              t power value
  --decay_learning_rate arg  Set Decay factor for learning_rate between passes
  --initial_t arg            initial t value


## Train VW

https://github.com/JohnLangford/vowpal_wabbit/wiki/Command-line-arguments

In [0]:
%%local
! head -n2 train.txt

1 |f did10000108 uid66d17a9ccd0e15 doid1126487 dow1 prm1 adid132815 pdid1227645 cid17018 advid331 US CA 807
-1 |f did10000108 uid66d17a9ccd0e15 doid1126487 dow1 prm1 adid133677 pdid1297868 cid17143 advid1919 US CA 807


In [0]:
%%local
! LD_LIBRARY_PATH=/usr/local/lib ./vw -d train.txt -b 24 -c -k --ftrl --passes 1 -f model --holdout_off --loss_function logistic --random_seed 42 --progress 8000000 

final_regressor = model
Enabling FTRL based optimization
Algorithm used: Proximal-FTRL
ftrl_alpha = 0.005
ftrl_beta = 0.1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
creating cache_file = train.txt.cache
Reading datafile = train.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.460771 0.460771      8000000      8000000.0  -1.0000  -2.5995       13
0.456128 0.451486     16000000     16000000.0   1.0000  -0.9527       13
0.453598 0.448538     24000000     24000000.0   1.0000  -1.6512       13
0.451978 0.447119     32000000     32000000.0  -1.0000  -1.5721       13
0.450767 0.445919     40000000     40000000.0  -1.0000  -2.4194       13
0.449798 0.444953     48000000     48000000.0  -1.0000  -0.9327       13
0.449048 0.444548     56000000     56000000.0   1.0000  -1.0814       13
0.448421 0.444031     64000000     64000000.0  -1.0000  -0.6477      

In [0]:
%%local
! LD_LIBRARY_PATH=/usr/local/lib ./vw -d test.txt -i model -t -k -p test_predictions.txt --progress 1000000 --link=logistic

only testing
predictions = test_predictions.txt
Enabling FTRL based optimization
Algorithm used: Proximal-FTRL
ftrl_alpha = 0.005
ftrl_beta = 0.1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = test.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
    n.a.     n.a.      1000000      1000000.0  unknown   0.0379       13
    n.a.     n.a.      2000000      2000000.0  unknown   0.0615       13
    n.a.     n.a.      3000000      3000000.0  unknown   0.5181       13
    n.a.     n.a.      4000000      4000000.0  unknown   0.0578       13
    n.a.     n.a.      5000000      5000000.0  unknown   0.2803       13
    n.a.     n.a.      6000000      6000000.0  unknown   0.1764       13
    n.a.     n.a.      7000000      7000000.0  unknown   0.1404       13
    n.a.     n.a.      8000000      8000000.0  unknown   0.1614      

In [0]:
%%local
def read_vw_predictions(p):
    y_pred = []
    with open(p, "r") as f:
        for line in f:
            y_pred.append(float(line.split()[0]))
    return np.array(y_pred)

In [0]:
%%local
import os
os.getcwd()

'/var/lib/jupyter'

In [0]:
%%local
!ls

clicks_test.csv.zip	      model			 Spark
clicks_train.csv.zip	      page_views.csv.zip	 Spark2
documents_categories.csv.zip  page_views_sample.csv.zip  test_predictions.txt
documents_entities.csv.zip    promoted_content.csv.zip	 test.txt
documents_meta.csv.zip	      PySpark			 train.txt
documents_topics.csv.zip      sample_submission.csv	 train.txt.cache
events.csv.zip		      sample_submission.csv.zip  vw


In [0]:
%%local
!unzip sample_submission.csv.zip

Archive:  sample_submission.csv.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


## Check VW test performance

In [0]:
%%local
! LD_LIBRARY_PATH=/usr/local/lib ./vw -d test.txt -i model -t -k -p test_predictions.txt --progress 1000000 --link=logistic

only testing
predictions = test_predictions.txt
Enabling FTRL based optimization
Algorithm used: Proximal-FTRL
ftrl_alpha = 0.005
ftrl_beta = 0.1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = test.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features

finished run
number of examples = 2
weighted example sum = 2.000000
weighted label sum = 0.000000
average loss = 0.966958
best constant = 0.000000
best constant's loss = 1.000000
total feature number = 22


In [0]:
%%local
! head -10 test_predictions.txt

0.198563
0.061126
0.106875
0.239665
0.180618
0.323399
0.267401
0.458192
0.407476
0.109474


In [0]:
%%local
!head -10 sample_submission.csv

display_id,ad_id
16874594,66758 150083 162754 170392 172888 180797
16874595,8846 30609 143982
16874596,11430 57197 132820 153260 173005 288385 289122 289915
16874597,137858 143981 155945 180965 182039 285834 305790 308836
16874598,67292 145937 250082 335632
16874599,91681 163776 173130 210516 213116 296295
16874600,2150 30682 57591 70529 114836 133050
16874601,14082 92003 118470 129490 140942 190713
16874602,131316 154918 268548 269017 281563


In [0]:
%%local
! head -10 test.txt

|f did16874673 uid18b8b693d40aae doid505235 dow4 prm1 adid207162 pdid1367996 cid23856 advid17 US OH 542"
|f did16874673 uid18b8b693d40aae doid505235 dow4 prm1 adid158923 pdid647796 cid2236 advid1309 US OH 542"
|f did16874673 uid18b8b693d40aae doid505235 dow4 prm1 adid107055 pdid208091 cid13829 advid372 US OH 542"
|f did16874673 uid18b8b693d40aae doid505235 dow4 prm1 adid91797 pdid1037379 cid11888 advid1347 US OH 542"
|f did16874705 uid92d6ee4f7f7df7 doid1778600 dow4 prm2 adid237858 pdid1596619 cid25733 advid1960 CA BC NULLGEO"
|f did16874705 uid92d6ee4f7f7df7 doid1778600 dow4 prm2 adid141374 pdid1324735 cid18099 advid1222 CA BC NULLGEO"
|f did16874705 uid92d6ee4f7f7df7 doid1778600 dow4 prm2 adid140897 pdid1322901 cid6202 advid83 CA BC NULLGEO"
|f did16874826 uid1adfb68d8975b9 doid1715349 dow5 prm2 adid124591 pdid1247375 cid15999 advid1754 NULLGEO NULLGEO NULLGEO"
|f did16874826 uid1adfb68d8975b9 doid1715349 dow5 prm2 adid124583 pdid1247333 cid15997 advid1754 NULLGEO NULLGEO NULLGEO"
|f

In [0]:
%%local
def util_key_value(row):
    x = row.split(' ')
    return x[1][3:], x[6][4:]

In [0]:
%%local
fo = open("test.txt", "r+")
print "Name of the file: ", fo.name
lines = fo.readlines()
util_key_value(lines[0])

Name of the file:  test.txt


('16874673', '207162')

In [0]:
fo = open("test.txt", "r+")
print "Name of the file: ", fo.name
lines = fo.readlines()
fo.close


In [0]:
%%local
!hdfs dfs -put test_predictions

In [0]:
%%local
fo_pred = open('test_predictions.txt', 'r+')
lines_pred = fo_pred.readlines()
fo_pred.close()

In [0]:
%%local
lines_pred[0]

'0.198563\n'

In [0]:
%%local
from tqdm import tqdm
from collections import defaultdict
dtas = defaultdict(list)

In [0]:
%%local
from tqdm import tqdm
dtas = defaultdict(list)
def did_to_aid_score():
    fo = open("test.txt", "r+")
    print "Name of the file: ", fo.name
    lines = fo.readlines()
    fo.close()
    fo_pred = open('test_predictions.txt', 'r+')
    lines_pred = fo_pred.readlines()
    fo_pred.close()
    for i in tqdm(range(len(lines))):
        x = util_key_value(lines[i])
        dtas[x[0]].append((lines_pred[i], x[1]))
did_to_aid_score()

Name of the file:  test.txt


100%|██████████| 32225162/32225162 [01:48<00:00, 296182.07it/s]


In [0]:
%%local
dtas['16874673']

[('0.198563\n', '207162'),
 ('0.061126\n', '158923'),
 ('0.106875\n', '107055'),
 ('0.239665\n', '91797')]

In [0]:
%%local
for k in tqdm(dtas.keys(), total=len(dtas.keys())):
    dtas[k] = sorted(dtas[k], key=lambda x: -float(x[0][:-2]))

100%|██████████| 6245533/6245533 [00:33<00:00, 186366.76it/s]


In [0]:
%%local
dtas['16874673']

[('0.239665\n', '91797'),
 ('0.198563\n', '207162'),
 ('0.106875\n', '107055'),
 ('0.061126\n', '158923')]

In [0]:
%%local
!head -10 sample_submission.csv

display_id,ad_id
16874594,66758 150083 162754 170392 172888 180797
16874595,8846 30609 143982
16874596,11430 57197 132820 153260 173005 288385 289122 289915
16874597,137858 143981 155945 180965 182039 285834 305790 308836
16874598,67292 145937 250082 335632
16874599,91681 163776 173130 210516 213116 296295
16874600,2150 30682 57591 70529 114836 133050
16874601,14082 92003 118470 129490 140942 190713
16874602,131316 154918 268548 269017 281563


In [0]:
%%local
import os
os.linesep

'\n'

In [0]:
%%local
!rm submission.csv

In [0]:
%%local
!ls

clicks_test.csv.zip	      model			 Spark
clicks_train.csv.zip	      page_views.csv.zip	 Spark2
documents_categories.csv.zip  page_views_sample.csv.zip  test_predictions.txt
documents_entities.csv.zip    promoted_content.csv.zip	 test.txt
documents_meta.csv.zip	      PySpark			 train.txt
documents_topics.csv.zip      sample_submission.csv	 train.txt.cache
events.csv.zip		      sample_submission.csv.zip  vw


In [0]:
%%local
with open('submission.csv', 'a') as the_file:
    the_file.write('display_id,ad_id' + os.linesep)
    for k in tqdm(dtas.keys(), total=len(dtas.keys())):
        row = str(k) + ',' + ' '.join(x[1] for x in dtas[k]) + os.linesep
        the_file.write(row)


100%|██████████| 6245533/6245533 [00:25<00:00, 246511.84it/s]


In [0]:
%%local
!head -20 submission.csv

display_id,ad_id
19774928,278053 43283 837 511486
20551788,153383 45192 528561 327451 470788 38762
20551789,332844 289971 537092 43260
21401181,95590 92760 135014 158345
21401180,458494 525506
21401183,93804 132386 290255 72073
21401182,212688 132785 215971 173400
21401185,161839 143173 65514 104898 12614 516721
21401184,540448 509890 377745 154113
21401187,87865 68989 183436 70017 256992 91879
21401186,3314 34162 370035 28243 288399 180923
21401189,546857 140448 255623 84898 301059 451239 180215 523502 228566
21401188,156270 38221 7033
21722259,153244 383090 401475
21722258,146272 176354 186979 173108 262434 184712
21722257,108253 517657 280430 56554
21722256,127542 42039 232800 232798 167294 167265
21722255,535997 6915 535690 520400 508614
21722254,11189 535997 76914 2175 154721


In [0]:
%%local
!kaggle competitions submit -c outbrain-click-prediction -f submission.csv -m 161

100%|██████████| 260M/260M [00:03<00:00, 81.0MB/s] 
Successfully submitted to Outbrain Click Prediction

In [0]:
%%local
!kaggle competitions submissions -c outbrain-click-prediction

500 - An internal server error occurred. Please ensure that your API client is up to date. If it is, please report a bug at github.com/Kaggle/kaggle-api


# WOW INTERACTIONS!

In [0]:
%%local
! LD_LIBRARY_PATH=/usr/local/lib ./vw -d train.txt -b 24 -c -k --ftrl --passes 1 -q ff -f model --holdout_off --loss_function logistic --random_seed 42 --progress 8000000

creating quadratic features for pairs: ff 
final_regressor = model
Enabling FTRL based optimization
Algorithm used: Proximal-FTRL
ftrl_alpha = 0.005
ftrl_beta = 0.1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
creating cache_file = train.txt.cache
Reading datafile = train.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.448838 0.448838      8000000      8000000.0  -1.0000  -2.8905       91
0.444975 0.441111     16000000     16000000.0   1.0000  -0.7125       91
0.442870 0.438660     24000000     24000000.0   1.0000  -1.6815       91
0.441525 0.437491     32000000     32000000.0  -1.0000  -1.7568       91
0.440504 0.436421     40000000     40000000.0  -1.0000  -2.5732       91
0.439656 0.435414     48000000     48000000.0  -1.0000  -0.9038       91
0.439016 0.435175     56000000     56000000.0   1.0000  -1.3737       91
0.438473 0.434670     6400

In [0]:
%%local
! LD_LIBRARY_PATH=/usr/local/lib ./vw -d test.txt -i model -t -k -p test_predictions.txt --progress 1000000 --link=logistic

creating quadratic features for pairs: ff 
only testing
predictions = test_predictions.txt
Enabling FTRL based optimization
Algorithm used: Proximal-FTRL
ftrl_alpha = 0.005
ftrl_beta = 0.1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = test.txt
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
    n.a.     n.a.      1000000      1000000.0  unknown   0.0249       91
    n.a.     n.a.      2000000      2000000.0  unknown   0.0352       91
    n.a.     n.a.      3000000      3000000.0  unknown   0.5012       91
    n.a.     n.a.      4000000      4000000.0  unknown   0.0730       91
    n.a.     n.a.      5000000      5000000.0  unknown   0.2351       91
    n.a.     n.a.      6000000      6000000.0  unknown   0.1709       91
    n.a.     n.a.      7000000      7000000.0  unknown   0.1616       91
    n.a.     n.a.      800

In [0]:
%%local
from tqdm import tqdm
dtas = defaultdict(list)
def did_to_aid_score():
    fo = open("test.txt", "r+")
    print "Name of the file: ", fo.name
    lines = fo.readlines()
    fo.close()
    fo_pred = open('test_predictions.txt', 'r+')
    lines_pred = fo_pred.readlines()
    fo_pred.close()
    for i in tqdm(range(len(lines))):
        x = util_key_value(lines[i])
        dtas[x[0]].append((lines_pred[i], x[1]))
did_to_aid_score()

Name of the file:  test.txt


100%|██████████| 32225162/32225162 [01:24<00:00, 381107.48it/s]


In [0]:
%%local
for k in tqdm(dtas.keys(), total=len(dtas.keys())):
    dtas[k] = sorted(dtas[k], key=lambda x: -float(x[0][:-2]))

100%|██████████| 6245533/6245533 [00:34<00:00, 182304.10it/s]


In [0]:
%%local
with open('submission2.csv', 'a') as the_file:
    the_file.write('display_id,ad_id' + os.linesep)
    for k in tqdm(dtas.keys(), total=len(dtas.keys())):
        row = str(k) + ',' + ' '.join(x[1] for x in dtas[k]) + os.linesep
        the_file.write(row)

100%|██████████| 6245533/6245533 [00:23<00:00, 260494.89it/s]


In [0]:
%%local
!kaggle competitions submit -c outbrain-click-prediction -f submission2.csv -m 161

100%|██████████| 260M/260M [00:03<00:00, 77.6MB/s] 
Successfully submitted to Outbrain Click Prediction

In [0]:
%%local
import os
os.system('kaggle competitions submissions -c outbrain-click-prediction')

0


In [0]:
# --ftrl  
# --ftrl_alpha (=0.005)        ftrl alpha parameter (option in ftrl)
# --ftrl_beta (=0.1)           ftrl beta patameter (option in ftrl)

In [0]:
%%local
import time

print("Before the sleep statement")
time.sleep(5)

Before the sleep statement


In [0]:
%%local
import os
count = 3
for alpha in [0.001, 0.01, 0.05]:
    for beta in [0.01, 0.05, 0.2]:
        os.system('LD_LIBRARY_PATH=/usr/local/lib ./vw -d train.txt -b 24 -c -k --ftrl --ftrl_alpha {alpha} --ftrl_beta {beta} --passes 1 -q ff -f model --holdout_off --loss_function logistic --random_seed 42 --progress 10000000'.format(alpha=alpha, beta=beta))
        os.system('LD_LIBRARY_PATH=/usr/local/lib ./vw -d test.txt -i model -t -k -p test_predictions.txt --progress 1000000 --link=logistic')
        dtas = defaultdict(list)
        did_to_aid_score()
        for k in dtas.keys():
            dtas[k] = sorted(dtas[k], key=lambda x: -float(x[0][:-2]))
        with open('submission' + str(count) + '.csv', 'a') as the_file:
            the_file.write('display_id,ad_id' + os.linesep)
            for k in tqdm(dtas.keys(), total=len(dtas.keys())):
                row = str(k) + ',' + ' '.join(x[1] for x in dtas[k]) + os.linesep
                the_file.write(row)
        os.system('kaggle competitions submit -c outbrain-click-prediction -f submission' + str(count) + '.csv -m 161')
        time.sleep(120)
        os.system('kaggle competitions submissions -c outbrain-click-prediction > tmp')
        print open('tmp', 'r').read()
        count += 1

In [0]:
%%local
!kaggle competitions submit -c outbrain-click-prediction -f submission3.csv -m 161

In [0]:
%%local
!kaggle competitions submit -c outbrain-click-prediction -f submission4.csv -m 161

100%|██████████| 260M/260M [00:03<00:00, 79.7MB/s] 
Successfully submitted to Outbrain Click Prediction

In [0]:
%%local
!kaggle competitions submit -c outbrain-click-prediction -f submission5.csv -m 161

100%|██████████| 260M/260M [00:03<00:00, 79.8MB/s] 
Successfully submitted to Outbrain Click Prediction

In [0]:
%%local
!kaggle competitions submit -c outbrain-click-prediction -f submission6.csv -m 161

100%|██████████| 260M/260M [00:03<00:00, 78.6MB/s] 
Successfully submitted to Outbrain Click Prediction

In [0]:
%%local
!kaggle competitions submit -c outbrain-click-prediction -f submission7.csv -m 161

100%|██████████| 260M/260M [00:03<00:00, 71.5MB/s] 
Successfully submitted to Outbrain Click Prediction

In [0]:
%%local
!kaggle competitions submit -c outbrain-click-prediction -f submission8.csv -m 161

100%|██████████| 260M/260M [00:03<00:00, 77.4MB/s] 
Successfully submitted to Outbrain Click Prediction

In [0]:
%%local
!kaggle competitions submissions -c outbrain-click-prediction

fileName         date                 description  status    publicScore  privateScore  
---------------  -------------------  -----------  --------  -----------  ------------  
submission8.csv  2020-03-19 20:16:28  161          complete  0.65134      0.65142       
submission7.csv  2020-03-19 20:14:05  161          complete  0.65117      0.65129       
submission6.csv  2020-03-19 20:03:47  161          complete  0.65110      0.65119       
submission5.csv  2020-03-19 19:59:10  161          complete  0.64106      0.64128       
submission4.csv  2020-03-19 19:57:23  161          complete  0.64127      0.64149       
submission3.csv  2020-03-19 19:55:02  161          complete  0.64134      0.64156       
